In [27]:
# ============================================================
# Notebook setup: run this before everything
# ============================================================
# -- Copied from lecture
%load_ext autoreload
%autoreload 1
%aimport util
from util import util
import numpy as np4
# from matplotlib import pyplot as plt
import pandas as pd

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 4)

data_folder = '../resources/dataset'
file_name = '1_gecco2019_water_quality.csv'
# Load the input data
data_path = f'{data_folder}/{file_name}'
data = pd.read_csv(data_path)
data['Time'] = pd.to_datetime(data['Time'])
data.set_index('Time', inplace=True)
data = data.drop(columns=["Unnamed: 0"])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
(data.index[1:] - data.index[:-1]).value_counts()

Time
0 days 00:01:00    132479
Name: count, dtype: int64

Luckily, the indices are already aligned to a sampling period of 1min.
We also see that no datapoint is missing because this would lead to inconsistent deltas
between datapoints.

Apart from the timestamps, we have the following columns:

In [26]:
", ".join(data.columns)

'Tp, pH, Cond, Turb, SAC, PFM, Event'

In the lectures, wen defined the alarm signal we strive to minimize:
$$
-\log f(x, \theta) \geq \varepsilon
$$